# Import data

In [17]:
import pandas as pd
import numpy as np

ins_df = pd.read_csv('data/instagram_data.csv')
ins_df

,imagename,postid,dataset,dataset.1,postdate,Contents,url,Q1_pertain_to_covid,Q2_cetegory,Q2A_Type of Human,...,Q8_threat_covid,Q9_susceptibility_covid,Q9A_Asian responsible for the covid,Q10_solution_present,Q11_recommended_solution,misinformation,Q12 Presence of conspiracy theory,Q12-Others,Q13. Image of plague doctor costume,Note
0,B-A5FIIIEKJ,5592,middle,2,3/21/20,#covid #covid2020 #covidvirus #virus #coronava...,https://www.instagram.com/p/B-A5FIIIEKJ/,1,1,1,...,1,1,2,1,1,0,2,99,2.0,Stay home
1,B-A74YQn_4a,5593,middle,2,3/21/20,Well this is the final mural of my trip in Aus...,https://www.instagram.com/p/B-A74YQn_4a/,1,1,1,...,1,1,2,2,99,0,2,99,2.0,NaN
2,B-AajnDp6GQ,5575,middle,2,3/21/20,Chegamos !!! Vão seguindo o movimento... Tem m...,https://www.instagram.com/p/B-AajnDp6GQ/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN
3,B-AarR4pUQA,5576,middle,2,3/21/20,😻😻😻😻😻,https://www.instagram.com/p/B-AarR4pUQA/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN
4,B-AE1y3HD-Z,5550,middle,2,3/21/20,EN MI DOMICILIO 🏡\r\r\r\r\n#quedateencasa\r\r\...,https://www.instagram.com/p/B-AE1y3HD-Z/,1,1,1,...,1,1,2,1,1,0,2,99,2.0,mask; Stay home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,CEZyheBgtMd,8402,second,3,8/27/20,#water #foryou #followforfollowback #photograp...,https://www.instagram.com/p/CEZyheBgtMd/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN
9644,CEZYoyRluKG,8352,second,3,8/27/20,#like4likes #20likes #tagforlikes #instalikes ...,https://www.instagram.com/p/CEZYoyRluKG/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN
9645,CEZz-solaUF,8403,second,3,8/27/20,🙈🥰😍👉🏽 @love_serie_karma #daancorona #daancoron...,https://www.instagram.com/p/CEZz-solaUF/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN
9646,CEZZ4P4j3rh,8354,second,3,8/27/20,💥🚨 𝗙𝗔𝗟𝗟 𝗜𝗦 𝗖𝗢𝗠𝗜𝗡𝗚 🚨💥⁣\r\r\r\r\r\n⁣\r\r\r\r\r\n...,https://www.instagram.com/p/CEZZ4P4j3rh/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN


In [66]:
from sklearn.utils import shuffle

anger_df = pd.read_csv('data/twitter/anger.tsv', sep='\t').drop(columns=['index', 'intensity'])
fear_df = pd.read_csv('data/twitter/fear.tsv', sep='\t').drop(columns=['index', 'intensity'])
joy_df = pd.read_csv('data/twitter/joy.tsv', sep='\t').drop(columns=['index', 'intensity'])
sadness_df = pd.read_csv('data/twitter/sadness.tsv', sep='\t').drop(columns=['index', 'intensity'])

emotion_df = pd.concat([anger_df, fear_df, joy_df, sadness_df])
emotion_df = shuffle(emotion_df)

emotion_df

,tweet,category
465,@delhi_lawyer @BhagwanUvach @ShivAroor sad . T...,sadness
781,@isthismike_p that nigga is horrible bro,fear
792,Don't think I'll hesitate to run you over. Las...,fear
264,@xandraaa5 @amayaallyn6 shut up hashtags are c...,anger
38,Today I reached 1000 subscribers on YT!! #happ...,joy
...,...,...
402,November #canola lost $5.50 to $464.20 per tonne.,sadness
48,I blame the whole season on Natalie! The seaso...,anger
525,Kudos to TC for benching Bruce. He isn't helpi...,sadness
835,How had Matty Dawson not scored there!!!!!,fear


# Feature Engineering

In [71]:
# TODO: feature engineering and data augmentation/cleaning. Since we are dealing with tweets (textual data), maybe explore TFIDF

# Model Definition

In [72]:
# TODO: define our machine learning model, from our discussion it we can try deep learning models

# Training

In [73]:
# TODO: train our model using the loaded data

# Performance Evaluation

In [75]:
# TODO: evaluate using accuracy to see how well our model is performing on the test set

# Prediction

In [76]:
# TODO: predict the submission data 